In [ ]:
%matplotlib inline
import scipy
import sympy
import numpy as np
import matplotlib.pyplot as plt
from modsim import*

condition = Condition(R = 5100,
                      L = 0.1,
                      C0 = 220 * (10 ** (-9)), 
                      V0 = 0,
                      f0 = 1073.02)




    


# look at differntiate method in scipy
# How to double integrate a function
# Do we solve using Inductor Voltage or Resistor Voltage
# How to include alternative current in our model


In [ ]:
def make_system(condition):
    
    unpack(condition)
    
    init = State(C = C0, f = f0, V = V0)
    
    
    
    
    return
                    
    
system = make_system(condition)

In [ ]:
def slope_func(state, t, system):
    
    Vac = (R * I) + (L * didt) + ((1 / C) * (integral of Idt))
    
    return

slope_func(init, 0, system)

In [ ]:
run_odient(system, slope_func)
